In [42]:
# Librerias

# Manipulacion 
import pandas as pd 
import geopandas as gpd
import re
from datetime import datetime

# Earth engine (descarga de imagenes)
import ee 

# Mapas interactivos
import folium

# Plot
from IPython.display import Image

In [2]:
# Autentificar Earth engine
# ee.Authenticate() # Esto solo debe ser ejecutado la primera vez y cada vez que el token expira

ee.Initialize()

In [3]:
# Definir area de estudio

# Coordenadas
lat = 21.8833333
lon = -102.3

# Punto de interes
poi = ee.Geometry.Point(lon, lat)

# Periodo de tiempo
start_date = "2021-11-01"
end_date = "2022-11-15"

In [4]:
# Descarga de imagenes Landsat 9
sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterBounds(poi)\
    .filterDate(start_date, end_date)

# Cantidad de imagenes obtenidas
print("Imagenes totales:", sentinel.size().getInfo())

Imagenes totales: 147


In [15]:
# Informacion de la primera imagen
sentinel.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32613',
   'crs_transform': [60, 0, 699960, 0, -60, 2500020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32613',
   'crs_transform': [10, 0, 699960, 0, -10, 2500020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32613',
   'crs_transform': [10, 0, 699960, 0, -10, 2500020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32613',
   'crs_transform': [10, 0, 699960, 0, -10, 2500020]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min':

In [5]:
# Nubosidad
print(sentinel.first().get("CLOUDY_PIXEL_PERCENTAGE").getInfo())

0.187117


In [6]:
# Porcentaje de agua
print(sentinel.first().get("WATER_PERCENTAGE").getInfo())

0.38235


In [49]:
# Fechas
print(datetime.strptime(re.findall("(?<=\_)\\d+(?=T)", sentinel.first().get("DATATAKE_IDENTIFIER").getInfo())[0], "%Y%m%d").date())

2021-11-03


In [11]:
# Bandas 
sentinel.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60']

In [12]:
# Parametros para preprocesamiento
parameters = {
    "min":7000,
    "max":16000,
    "dimensions":800, # Tamaño cuadrado en pixeles
    "bands": ["B2", "B3", "B4"] # (R, G, B)
}

In [58]:
# Imagenes en lista
sentinel_list = sentinel.toList(sentinel.size())

# Funcion para mostrar cada imagen
def display_img(ids, parameters, ndvi_display = False):

    for i in ids:
    
        # Fecha de captura
        date = datetime.strptime(re.findall("(?<=\_)\\d+(?=T)", ee.Image(sentinel_list.get(i)).get("DATATAKE_IDENTIFIER").getInfo())[0], "%Y%m%d").date()

        # Nubosidad
        cloud = ee.Image(sentinel_list.get(i)).get("CLOUDY_PIXEL_PERCENTAGE").getInfo()

        # Porcentaje de vegetación
        veg = ee.Image(sentinel_list.get(i)).get("VEGETATION_PERCENTAGE").getInfo()

        # Porcentaje de agua
        water = ee.Image(sentinel_list.get(i)).get("WATER_PERCENTAGE").getInfo()

        # Informacion de la imagen
        print("Imagen #", i, date, "Nubosidad:", cloud, "Porcentaje de agua:", water)

        if ndvi_display == True:
            display(Image(url = ee.Image(sentinel_list.get(i)).normalizedDifference(["B5", "B4"]).getThumbURL(parameters)))

        else:
            # Plot
            display(Image(url = ee.Image(sentinel_list.get(i)).getThumbURL(parameters)))
        
# Funcion para crear data frame con info de las imagenes
def img_info(ids):
    data = []
    for i in ids:

        # Fecha de captura
        date = datetime.strptime(re.findall("(?<=\_)\\d+(?=T)", ee.Image(sentinel_list.get(i)).get("DATATAKE_IDENTIFIER").getInfo())[0], "%Y%m%d").date()

        # Nubosidad
        cloud = ee.Image(sentinel_list.get(i)).get("CLOUDY_PIXEL_PERCENTAGE").getInfo()

        # Porcentaje de vegetación
        veg = ee.Image(sentinel_list.get(i)).get("VEGETATION_PERCENTAGE").getInfo()
        
        # Porcentaje de agua
        water = ee.Image(sentinel_list.get(i)).get("WATER_PERCENTAGE").getInfo()

        # Informacion de la imagen
        print("Imagen #", i, date, "Nubosidad:", cloud,"Porcentaje de vegetación:", veg, "Porcentaje de agua:", water)
        
        image_data = [i, date, cloud, veg, water]
        data.append(image_data)
    
    return data

In [57]:
# DF de informacion
sentinel_df = pd.DataFrame(img_info(range(sentinel.size().getInfo())), columns = ["img_id", "date", "cloud_cover", "vegetation_percentage", "water_percentage"])

Imagen # 0 2021-11-03 00:00:00 Nubosidad: 0.187117 Porcentaje de agua: 0.38235
Imagen # 1 2021-11-05 00:00:00 Nubosidad: 0.576502 Porcentaje de agua: 0.273206
Imagen # 2 2021-11-10 00:00:00 Nubosidad: 54.052523 Porcentaje de agua: 0.052212
Imagen # 3 2021-11-13 00:00:00 Nubosidad: 35.121469 Porcentaje de agua: 0.224204
Imagen # 4 2021-11-15 00:00:00 Nubosidad: 0.24273 Porcentaje de agua: 0.233653
Imagen # 5 2021-11-18 00:00:00 Nubosidad: 1.858585 Porcentaje de agua: 0.335685
Imagen # 6 2021-11-20 00:00:00 Nubosidad: 0.241683 Porcentaje de agua: 0.260036
Imagen # 7 2021-11-23 00:00:00 Nubosidad: 0.945583 Porcentaje de agua: 0.384063
Imagen # 8 2021-11-25 00:00:00 Nubosidad: 0.478377 Porcentaje de agua: 0.225956
Imagen # 9 2021-11-28 00:00:00 Nubosidad: 8.957708 Porcentaje de agua: 0.253872
Imagen # 10 2021-11-30 00:00:00 Nubosidad: 0.299219 Porcentaje de agua: 0.237274
Imagen # 11 2021-12-03 00:00:00 Nubosidad: 2.284935 Porcentaje de agua: 0.344048
Imagen # 12 2021-12-05 00:00:00 Nubosi

In [73]:
# Imagenes recolectadas
parameters = {
    "min":0, # 0 bueno
    "max":3000, # 3000 bueno
    "dimensions":800, # Tamaño cuadrado en pixeles
    "bands": ["B4", "B3", "B2"] # (R, G, B)
}

display_img([88], parameters = parameters)

Imagen # 88 2022-06-18 Nubosidad: 16.573383 Porcentaje de agua: 0.099891


In [74]:
# Data frame con  informacion de las imagenes
sentinel_df.head() # Con esto se puede filtrar por la nubosidad, la cual parece ser buena por debajo de 10

,img_id,date,cloud_cover,water_percentage
0,0,2021-11-03,0.187117,0.382350
1,1,2021-11-05,0.576502,0.273206
2,2,2021-11-10,54.052523,0.052212
3,3,2021-11-13,35.121469,0.224204
4,4,2021-11-15,0.242730,0.233653


## Filtrado de imagenes

In [76]:
# Seleccionar las imagenes que tengan una nubosidad por debajo de 10
ids_noClouds = sentinel_df.img_id[sentinel_df.cloud_cover < 10]
len(ids_noClouds) # 90 Imagenes 

90

In [80]:
# Definir region de interes
roi = poi.buffer(5000) # Metros = 5 km

In [81]:
# Nuevos parametros para región de interés
parameters_roi = {
    "min":0,
    "max":3000,
    "dimensions":800, # Tamaño cuadrado en pixeles
    "bands": ["B4", "B3", "B2"], # (R, G, B)
    "region":roi # Region de interes
}

In [82]:
display_img([0], parameters_roi)

Imagen # 0 2021-11-03 Nubosidad: 0.187117 Porcentaje de agua: 0.38235


In [83]:
# Imagenes con zoom en la ciudad de Ags
display_img(ids_noClouds, parameters = parameters_roi)

Imagen # 0 2021-11-03 Nubosidad: 0.187117 Porcentaje de agua: 0.38235


Imagen # 1 2021-11-05 Nubosidad: 0.576502 Porcentaje de agua: 0.273206


Imagen # 4 2021-11-15 Nubosidad: 0.24273 Porcentaje de agua: 0.233653


Imagen # 5 2021-11-18 Nubosidad: 1.858585 Porcentaje de agua: 0.335685


Imagen # 6 2021-11-20 Nubosidad: 0.241683 Porcentaje de agua: 0.260036


...

Imagen # 55 2022-03-25 Nubosidad: 0.000241 Porcentaje de agua: 0.133161


Imagen # 56 2022-03-28 Nubosidad: 3.1e-05 Porcentaje de agua: 0.209907


Imagen # 57 2022-03-30 Nubosidad: 0.010731 Porcentaje de agua: 0.09509


Imagen # 58 2022-04-02 Nubosidad: 5.1e-05 Porcentaje de agua: 0.19865


KeyboardInterrupt: 

## NDVI

$NDVI = \frac{NIR - RED}{NIR + RED}$

* Saludable = Valores altos
* No saludable = Valores bajos

`ndvi = image.normalizedDifference(["B5", "B4"])`

In [84]:
# Parametros para el analisis NDVI
palette = ["red", "yellow", "green"]

ndvi_parameters = {
    "min":0, # Este color sera rojo - no saludable
    "max":0.4, # Lo que sea mayor sera verde - saludable
    "dimensions":512,
    "palette":palette,
    "region":roi
}

In [85]:
# NDVI index por imagen
display_img(ids_noClouds, ndvi_parameters, ndvi_display = True)

Imagen # 0 2021-11-03 Nubosidad: 0.187117 Porcentaje de agua: 0.38235


Imagen # 1 2021-11-05 Nubosidad: 0.576502 Porcentaje de agua: 0.273206


Imagen # 4 2021-11-15 Nubosidad: 0.24273 Porcentaje de agua: 0.233653


KeyboardInterrupt: 